# Demo for runtime client

## Starting a client

First, we need to start a runtime and a runtime client.

In [2]:
import sys

sys.path.append("..")

from toolmaker.runtime.client import DockerRuntimeClient

client = DockerRuntimeClient.create(
    name="toolmaker-client", reuse_existing=False, build=True
)

2025-02-19 14:01:08.604 | DEBUG    | toolmaker.runtime.client:build_image:161 - Building image toolmaker-runtime:latest using Docker BuildKit
2025-02-19 14:01:16.299 | DEBUG    | toolmaker.runtime.client:build_image:172 - Error printing build output: {'aux': {'ID': 'sha256:06e1bbe391ca5d4504497b9789dde0a4dfde0333d3ac79adc5cb1bad93edbf4a'}}
2025-02-19 14:01:16.301 | INFO     | toolmaker.runtime.client:build_image:174 - Built image toolmaker-runtime:latest using Docker BuildKit
2025-02-19 14:01:16.303 | DEBUG    | toolmaker.runtime.client:_start_container:205 - Starting container with GPUs ['0'] on port 8000


Step 1/15 : FROM python:3.12
 ---> 1a0aa7c771b9
Step 2/15 : ENV HOST=0.0.0.0
 ---> Using cache
 ---> 1367f556dfe6
Step 3/15 : ENV PORT=8000
 ---> Using cache
 ---> b7c458369eeb
Step 4/15 : RUN mkdir -p /toolmaker
 ---> Using cache
 ---> e7111a458120
Step 5/15 : COPY pyproject.toml /toolmaker/pyproject.toml
 ---> Using cache
 ---> 5303632c7a84
Step 6/15 : COPY uv.lock /toolmaker/uv.lock
 ---> Using cache
 ---> b4a110547937
Step 7/15 : COPY toolmaker /toolmaker/toolmaker
 ---> Using cache
 ---> 5750bf88fca0
Step 8/15 : COPY scripts/toolmaker_function_runner.py /toolmaker/toolmaker_function_runner.py
 ---> Using cache
 ---> 6031d295b94f
Step 9/15 : COPY scripts/subprocess_utils.py /toolmaker/subprocess_utils.py
 ---> Using cache
 ---> 9ab15b730b43
Step 10/15 : RUN python -m pip install uv &&     cd /toolmaker &&     uv venv &&     uv sync
 ---> Using cache
 ---> 9cf9f64fb3bf
Step 11/15 : RUN mkdir -p /workspace
 ---> Using cache
 ---> b40f813f2629
Step 12/15 : WORKDIR /workspace
 ---> Usi

2025-02-19 14:01:17.011 | INFO     | toolmaker.runtime.client:_start_container:220 - Started runtime client toolmaker-client on port 8000 from image toolmaker-runtime:latest


The cell above performs two tasks:
1. It starts a new runtime in a new Docker container named `"toolmaker-client"`.
2. It starts a runtime client in order to interact with that runtime programatically.

Specifically, the first step is equivalent to:
```bash
uv run python -m toolmaker.runtime.create --name toolmaker-client

# Or, if you want to use docker commands directly, you could instead run:
docker buildx build -f runtime/Dockerfile -t toolmaker-client .
docker run -it -p 8000:8000 --rm toolmaker-client
```

The client will be running on `localhost:8000` (the Docker container's port 8000 will be mounted to local port 8000).
To check if the client is running, see if the docs are available at [http://localhost:8000/docs](http://localhost:8000/docs).

## Using the client
First, let's programmatically check that the client is alive.

In [3]:
client.is_alive()

True

Let's create an action to run on the client.

In [6]:
from toolmaker.actions import RunBashCommand

command = RunBashCommand(command="pwd", env=[])
command

RunBashCommand(command='pwd', env=[], reasoning='')

Now, run the action:

In [7]:
observation = client.execute(command)
print(observation.content)

/workspace



Let's try some more actions.

In [8]:
from toolmaker.actions import WriteFile

client.execute(
    WriteFile(path="test.txt", content="Hello, world!", description="A test file")
)

FileWriteObservation(content=None, filename='test.txt')

In [9]:
from toolmaker.actions import ListDirectory

client.execute(ListDirectory(path=""))

ListDirectoryObservation(content=['/workspace/test.txt'])

In [10]:
from toolmaker.actions import ReadFile

client.execute(ReadFile(path="test.txt"))

FileReadObservation(content='Hello, world!')

In [12]:
from toolmaker.actions import Browse

client.execute(Browse(url="https://www.google.com"))

BrowseObservation(content='Google Suche [Bilder](https://www.google.com/imghp?hl=de&tab=wi) [Maps](https://maps.google.de/maps?hl=de&tab=wl) [Play](https://play.google.com/?hl=de&tab=w8) [YouTube](https://www.youtube.com/?tab=w1) [News](https://news.google.com/?tab=wn) [Gmail](https://mail.google.com/mail/?tab=wm) [Drive](https://drive.google.com/?tab=wo) [Mehr »](https://www.google.de/intl/de/about/products?tab=wh) [Webprotokoll](http://www.google.de/history/optout?hl=de) | [Einstellungen](/preferences?hl=de) | [Anmelden](https://accounts.google.com/ServiceLogin?hl=de&passive=true&continue=https://www.google.com/&ec=GAZAAQ) [Erweiterte Suche](/advanced_search?hl=de&authuser=0) [Werbeprogramme](/intl/de/ads/) [Unternehmensangebote](/services/) [Éber Google](/intl/de/about.html) [Google.de](https://www.google.com/setprefdomain?prefdom=DE&prev=https://www.google.de/&sig=K_xbS_LBKHQSTSR-6DP48PQcaEwME%3D) © 2025 - [Datenschutzerklðrung](/intl/de/policies/privacy/) - [Nutzungsbedingungen](/

## Stopping the client
To stop and delete the client after completing this notebook, run:
```bash
docker stop toolmaker-client
docker rm toolmaker-client
```